In [9]:
import tensorflow as tf
from nets.dep_network import depth_estimation
from nets.pose_network import pose_est
from losses import *
#from utils import *
from data_loader import data_loader_with_batch,load_intrinsic

In [10]:
epsilon=0.0000001
@tf.function
def train_step(de,pe,image_left,image_right,intrinsic):
    """
    de is depth estimation model
    pe is pose estimation model
    intrinsic shape [batch,3,3]
    """
    B = 0.54
    f = 721/1242
    with tf.GradientTape(watch_accessed_variables=False) as tape_de:
        tape_de.watch(de.trainable_variables)
        dcx7_right,dep_right = de(image_right)
        dcx7_left,dep_left = de(image_left)
        loss_depth = dept_est_loss(dep_left,dep_right,image_left,image_right)
    
    with tf.GradientTape(watch_accessed_variables=False) as tape_pe:
        tape_pe.watch(pe.trainable_variables)
        t_left,rot_left = pe(image_left)

        t_right,rot_right = pe(image_right)
        disp = (dep_right[:,:,:,0]+epsilon)
        depth = (B*f)/disp
        pk = tf.slice(image_right,[0,0,0,0],[-1,-1,-1,3])
        pk1 = tf.slice(image_right,[0,0,0,3],[-1,-1,-1,3])
        loss_pose_total = pose_est_loss(rot_right,rot_left,t_left,t_right,pk,pk1,intrinsic,depth)
        #tf.print("loss_pose",loss_pose)
    gradients_de = tape_de.gradient(loss_depth, de.trainable_variables)
    gradients_pe = tape_pe.gradient(loss_pose_total, pe.trainable_variables)
    return loss_depth,gradients_de,loss_pose_total,gradients_pe

In [11]:
train_ds = []
train_ps = []
#de = depth_estimation()
#pe = pose_est()
batch = 2

##for training images dataset
path_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training"
left_img_dir = path_dir+"/image_2/"
right_img_dir = path_dir+"/image_3/"
img_width = 416
img_height = 128
batch = 2

##for intrinsic the dir is 
calib_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/data_scene_flow_calib/training/calib_cam_to_cam/"
internsic_tensor = load_intrinsic(calib_dir,batch)

train_ds = data_loader_with_batch(left_img_dir,right_img_dir,img_width,img_height,batch,number_of_data=100)

In [ ]:
de = depth_estimation()
pe = pose_est()
EPOCHS = 6
optimizer_de = tf.keras.optimizers.Adadelta(0.001)
optimizer_pe = tf.keras.optimizers.Adadelta(0.001)

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    for images_left,images_right in train_ds:
        ld,grad_de,lp,grad_pe = train_step(de,pe,images_left,images_right,internsic_tensor)
        ##ld is the loss in depth
        ##lp is the loss in pose
        #print("initial_loss",ld.numpy())
        optimizer_de.apply_gradients(zip(grad_de, de.trainable_variables))
        optimizer_pe.apply_gradients(zip(grad_pe, pe.trainable_variables))
        #print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),ld.numpy()))

    #grad = tf.convert_to_tensor(grad)
    print("epochs : ",epoch)
    print("loss depth",ld.numpy())
    print("loss pose",lp.numpy())
    #print("grad",grad)#.numpy())
    #print("loss pose",lp.numpy())

epochs :  0
loss depth 1.3298216
loss pose 0.32866976
epochs :  1
loss depth 1.301612
loss pose 0.3150618


In [5]:
_,dep = de(images_right)